# **DeepLandforms**

Author: giacomo.nodjoumi@hyranet.info - g.nodjoumi@jacobs-university.de

## DeepLandforms Segmentation

With this notebook, users can use custom trained models for instance segmentation models on custom dataset of georeferenced images.
The output consist of a folder containing:
* Source Images in which at least one detection occurred
* Label file in COCO json format for each image
* Geopackage containing a single layer with image name, confidence leve, class.

## Usage

* Put or link the dataset into the **DeepLandforms** *.env* file
* Run docker-compose up
* Edit the *configs* section by editing the following parameters:
------------------------------------------------------------------
| **Parameter** | **Function** | **Common Values** |
| ---- | ---- | ---- |
| **batch_size** | N° of images to be processed at once | Depending on VRAM and image size, up to 8 per 8GB VRAM |
| **geopackage_name** |  Name of the final geopackage |  |
| **proj_geopackage_name** | Name of the final geopackage in custom projection | |
| **model_path** | local path and name of the model  | it should start with /pre-trained_models/ |
| **model_yaml** | Model Architecture | MASK-R-CNN in this work | EDIT according to trained model selected |
| **dst_crs** | CRS of the geopackage | provide as WKT or proj4 |

------------------------------------------------------------------
Then just execute the notebook and monitor the training in **Tensorboard** container.

## Funding
*This study is within the Europlanet 2024 RI and EXPLORE project, and it has received funding from the European Union’s Horizon 2020 research and innovation programme under grant agreement No 871149 and No 101004214.*

In [1]:
import cv2
from datetime import datetime
import detectron2
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.data.catalog import Metadata
import numpy as np
import os
import geopandas as gpd
import pandas as pd
import psutil
from pyproj import CRS
#from pycocotools import mask
import random
import rasterio as rio
from rasterio.plot import reshape_as_image
import shutil
import torch
from tqdm import tqdm
from utils.GenUtils import get_paths
from utils.detectron_utils import CustomPredictor
from utils.geoshape_utils import parallel_funcs, chunk_creator, mask2shape, pred2coco, pred2shape, crs_validator

In [2]:
print(torch.__version__)
torch.cuda.is_available()
torch.cuda.get_device_name()

1.9.0+cu111


'NVIDIA GeForce RTX 2070 with Max-Q Design'

## CONFIGURATION - edit befor run

In [3]:
batch_size = 8
image_path = '../data'
geopackage_name = '/Inferred_Shapes.gpkg' ## Example for HiRISE 
proj_geopackage_name = '/Inferred_Shapes_projected.gpkg' ## Example for HiRISE 
model_path = '../pre-trained_model/model_final.pth'
model_yaml = "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml" ## EDIT according to trained model selected
#src_crs = CRS.from_user_input('PROJCRS["Equirectangular MARS", BASEGEOGCRS["GCS_MARS",DATUM["unnamed",ELLIPSOID["unnamed",3393833.2607584,0,LENGTHUNIT["metre",1,ID["EPSG",9001]]]],PRIMEM["Reference meridian",0,ANGLEUNIT["degree",0.0174532925199433,ID["EPSG",9122]]]],CONVERSION["unnamed",METHOD["Equidistant Cylindrical",ID["EPSG",1028]],PARAMETER["Latitude of natural origin",20,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8801]],PARAMETER["Longitude of natural origin",180,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8802]],PARAMETER["Latitude of 1st standard parallel",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8823]],PARAMETER["False easting",0,LENGTHUNIT["metre",1],ID["EPSG",8806]],PARAMETER["False northing",0,LENGTHUNIT["metre",1],ID["EPSG",8807]]],CS[Cartesian,2],AXIS["easting",east,ORDER[1],LENGTHUNIT["metre",1,ID["EPSG",9001]]],AXIS["northing",north,ORDER[2],LENGTHUNIT["metre",1,ID["EPSG",9001]]]]')
#dst_crs = CRS.from_user_input('PROJCS["Equirectangular MARS",GEOGCS["GCS_MARS",DATUM["unnamed",SPHEROID["unnamed",3395582.0270805,0]],PRIMEM["Reference meridian",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Equirectangular"],PARAMETER["latitude_of_origin",10],PARAMETER["central_meridian",180],PARAMETER["standard_parallel_1",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]')
dst_crs = CRS.from_wkt('PROJCRS["Equirectangular MARS", BASEGEOGCRS["GCS_MARS", DATUM["unnamed", ELLIPSOID["unnamed",3396190,0, LENGTHUNIT["metre",1,  ID["EPSG",9001]]]], PRIMEM["Reference meridian",0, ANGLEUNIT["degree",0.0174532925199433,		ID["EPSG",9122]]]], CONVERSION["Equidistant Cylindrical", METHOD["Equidistant Cylindrical", ID["EPSG",1028]], PARAMETER["Latitude of 1st standard parallel",0, ANGLEUNIT["degree",0.0174532925199433], ID["EPSG",8823]], PARAMETER["Longitude of natural origin",180, ANGLEUNIT["degree",0.0174532925199433], ID["EPSG",8802]], PARAMETER["False easting",0, LENGTHUNIT["metre",1], ID["EPSG",8806]], PARAMETER["False northing",0, LENGTHUNIT["metre",1], ID["EPSG",8807]]], CS[Cartesian,2], AXIS["easting",east, ORDER[1], LENGTHUNIT["metre",1, ID["EPSG",9001]]], AXIS["northing",north, ORDER[2], LENGTHUNIT["metre",1, ID["EPSG",9001]]]]')
#dst_crs = CRS.from_user_input('GEOGCRS["GCS_Mars_2000_Sphere", DATUM["Mars_2000_(Sphere)", ELLIPSOID["Mars_2000_Sphere_IAU_IAG",3396190,0, LENGTHUNIT["metre",1]], ID["ESRI",106971]], PRIMEM["Reference_Meridian",0, ANGLEUNIT["Degree",0.0174532925199433]], CS[ellipsoidal,2], AXIS["longitude",east, ORDER[1], ANGLEUNIT["Degree",0.0174532925199433]], AXIS["latitude",north, ORDER[2], ANGLEUNIT["Degree",0.0174532925199433]]]')

In [4]:

os.getcwd()

'/home/user/DeepLandforms'

In [5]:
out_dir = image_path+'/outputs'
os.makedirs(out_dir, exist_ok=True)
class_file = os.path.dirname(model_path)+'/trained_classes.csv'
class_df = pd.read_csv(class_file)
classes = class_df[class_df.columns[0]].tolist()
meta = Metadata()
meta.set(thing_classes=classes)

namespace(thing_classes=['Type-3',
                         'Type-4',
                         'Type-1a',
                         'Crater',
                         'Type-1b',
                         'Type-2b',
                         'Type-2a'])

In [6]:
images = get_paths(image_path,'tiff')
src_crs = CRS.from_wkt(rio.open(image_path+'/'+images[0]).crs.to_wkt())


In [7]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(model_yaml))
cfg.MODEL.ROI_HEADS.NUM_CLASSES =  len(classes)
cfg.MODEL.WEIGHTS = model_path
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.8

In [8]:
cols = ['Name','Class','Score']
dst_gpkg = out_dir+'/Inferred_Shapes.gpkg'
proc_csv = out_dir+'/Processed.csv'
try:
    geoshapes = gpd.read_file(dst_gpkg)
except Exception as e:
    print(e)
    #geoshapes = gpd.GeoDataFrame(columns=cols)
    pass
try:
    proc_df = pd.read_csv(proc_csv)
except Exception as e:
    print(e)
    proc_df = pd.DataFrame(columns=['Name','Detections'])
    pass

../data/outputs/Inferred_Shapes.gpkg: No such file or directory
[Errno 2] No such file or directory: '../data/outputs/Processed.csv'


In [9]:
chunks = list(chunk_creator(images,batch_size))
len(chunks)

1

In [10]:
CP = CustomPredictor(cfg)
JOBS=psutil.cpu_count(logical=False)
with tqdm(total=len(images),
             desc = 'Generating Images',
             unit='File') as pbar:
    start = datetime.now()
    gdf_list = []
    for d in range(len(chunks)):
        chunk = list(chunks[d])

        lambda_f = lambda element:element not in proc_df['Name'].to_list()
        filtered = filter(lambda_f, chunk)

        chunk = list(filtered)
        if len(chunk)>0:
            
            data_dict = [{'Name': ii, 'Detections': np.nan} for ii in chunk]
            tmp_df = pd.DataFrame.from_dict(data_dict)
            paths = [image_path+'/'+ele for ele in chunk]
            open_images = [rio.open(img_path) for img_path in paths]
            imgs = [reshape_as_image(image.read()) for image in open_images]
            predictions = CP(imgs)
            masks = predictions[0]['instances'].pred_masks.cpu().numpy()
            if len(masks)>0:
                detections = []
                for i in range(len(predictions)):
                    gdf = pred2shape(predictions[i], paths[i], open_images[i],classes, JOBS, out_dir, i)    
                    gdf_list.append(gdf)
                    #geoshapes = geoshapes.append(gdf, ignore_index=True)
                    #try:
                    #    reproj_gdf = crs_validator(geoshapes, gdf)
                    #    geoshapes = gpd.GeoDataFrame(pd.concat([geoshapes,reproj_gdf],ignore_index=True),crs=src_crs)
                    #except:
                    #    geoshapes = gdf.copy()
                    #if geoshapes.crs != gdf.crs:
                    #    gdf.to_crs(geoshapes.crs)
                    #geoshapes = gpd.GeoDataFrame(pd.concat([geoshapes,reproj_gdf],ignore_index=True),crs=src_crs)
                    #geoshapes.append(gdf)
                    #cv2.imwrite(out_dir,open_images[i])
                    shutil.copy(paths[i], out_dir+'/'+os.path.basename(paths[i]))
                    #gdf.to_file(dst_gpkg.split('.gpkg')[0]+str(i)+'.gpkg', driver='GPKG', crs=src_crs)     
                    #geoshapes.crs = src_crs
                    #detections.append(len(geoshapes))
                    detections.append(len(gdf))
                    #tmp_df['Detections']=len(geoshapes)
                    tmp_df['Detections']=len(gdf)
                    proc_df = proc_df.append(tmp_df,ignore_index=True)
                    proc_df.to_csv(proc_csv, index=False)
            else:
                tmp_df['Detections'] = 0
                proc_df = proc_df.append(tmp_df,ignore_index=True)
                proc_df.to_csv(proc_csv, index=False)

        pbar.update(batch_size)        
        
    stop = datetime.now()
    print(stop-start)

The checkpoint state_dict contains keys that are not used by the model:
  pixel_mean
  pixel_std
Generating Images:   0%|          | 0/2 [00:00<?, ?File/s]/usr/local/lib/python3.9/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
/usr/local/lib/python3.9/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/

0:00:04.717602


In [14]:
try:
    geoshapes.crs
except:
    geoshapes = gdf_list[0]

In [29]:
init_crs = gdf_list[0].crs
for geo in gdf_list:
    if geo.crs != init_crs:
        geo = geo.to_crs(init_crs)
#    geoshapes.append(geo, ingore_index=True)
    geoshapes = gpd.GeoDataFrame(pd.concat([geoshapes,geo],ignore_index=True),crs=init_crs)

In [30]:
geoshapes

,Name,Class,Score,geometry
0,ESP_012600_1655_RED_resized_5.0m.tiff,Crater,0.997397,"POLYGON ((3505082.829 -852660.064, 3505152.848..."
1,ESP_012600_1655_RED_resized_5.0m.tiff,Crater,0.996076,"POLYGON ((3504667.717 -849234.887, 3504712.729..."
2,ESP_012600_1655_RED_resized_5.0m.tiff,Type-3,0.993024,"POLYGON ((3502267.066 -840029.411, 3502562.146..."
3,ESP_012600_1655_RED_resized_5.0m.tiff,Crater,0.984109,"POLYGON ((3505342.900 -854160.141, 3505407.917..."
4,ESP_012600_1655_RED_resized_5.0m.tiff,Crater,0.951020,"POLYGON ((3505287.885 -853520.108, 3505362.905..."
5,ESP_012600_1655_RED_resized_5.0m.tiff,Crater,0.926981,"POLYGON ((3504727.733 -849784.915, 3504747.738..."
6,ESP_012600_1655_RED_resized_5.0m.tiff,Crater,0.921223,"POLYGON ((3507213.407 -849929.923, 3507258.419..."
7,ESP_012600_1655_RED_resized_5.0m.tiff,Crater,0.884942,"POLYGON ((3504752.740 -850289.941, 3504802.753..."
8,ESP_012600_1655_RED_resized_5.0m.tiff,Crater,0.809708,"POLYGON ((3505207.863 -853135.088, 3505277.882..."
9,ESP_012600_1655_RED_resized_5.0m.tiff,Crater,0.997397,"POLYGON ((3505082.829 -852660.064, 3505152.848..."
